In [ ]:
# Multi-core version

from tqdm.notebook import tqdm

datasets = [
#  Dataset.IRIS2D,
  Dataset.WINE,
  Dataset.PIMA,
  Dataset.BC
]

classifiers = [
  Classifier.GNB,
  Classifier.MLP,
  Classifier.RF
]

kernel_widths = [
  0.1,
  # 0.2,
  0.3,
  # 0.4,
  0.5,
  # 0.6,
  # 0.7,
  # 0.8,
  # 0.9,
  1.0,
  # 1.5,
  # 2.0,
  # 2.5,
  # 3.0,
  # 3.5,
  lambda n: np.sqrt(n) * .75,
  4.0
]

columns = pd.MultiIndex.from_product([datasets, classifiers])

def faithfulness(options):
  dataset, classifier, kernel_width = options
  data = dataset.instance
  Xtr = data.data
  clf = classifier.instance
  clf.fit(Xtr, data.target)

  points, points2 = do_experiment(Xtr, clf, kernel_width, plot_sample_data=False)
  score_lemon = np.mean(points[:,1])
  score_lime = np.mean(points2[:,1])

  return score_lime, score_lemon


import contextlib
import joblib
from tqdm.notebook import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()


from itertools import product
from joblib import delayed, Parallel, cpu_count

parameters = list(product(datasets, classifiers, kernel_widths))


with tqdm_joblib(tqdm(total=len(parameters))) as progress_bar:
  result = Parallel(n_jobs=cpu_count())(delayed(faithfulness)(p) for p in parameters)


_RemoteTraceback                          Traceback (most recent call last)
_RemoteTraceback:
"""
Traceback (most recent call last):
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\process_executor.py", line 463, in _process_worker
    r = call_item()
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 598, in <listcomp>
    return [func(*args, **kwargs)
  File "C:\Users\Acer\AppData\Local\Temp\ipykernel_12720\403947192.py", line 47, in faithfulness
  File "C:\Users\Acer\AppData\Local\Temp\ipykernel_12720\556752134.py", line 71, in do_experiment
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\lime\lime_tabular.py", line 452, in explain_instance
    ret_exp.score, ret_exp.local_pred) = self.base.explain_instance_with_data(
  File "c:\Users\Acer\XAI_IDS\IDS\pwla_lime\pwla_line_3d.py", line 94, in explain_instance_with_data
    easy_model.fit(
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py", line 1130, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py", line 854, in fit
    X, y, X_offset, y_offset, X_scale = _preprocess_data(
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py", line 284, in _preprocess_data
    X_offset = np.average(X, axis=0, weights=sample_weight)
  File "<__array_function__ internals>", line 180, in average
  File "c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py", line 547, in average
    raise ZeroDivisionError(
ZeroDivisionError: Weights sum to zero, can't be normalized
"""